# Key Concept Tasks
The main building block of task_chain is the task object. A Task contains key information as: Name, Description, Typ, Inputs/Outputs, its relations to other tasks and states. A Task can have several types which mainly represent the hierarchy of the task within the big picture.
We differentiate between the following Tasks:
- **Project**: A high-level Goal.
- **Pipeline**: A pipeline is a high-level task with the goal to achieve a milestone within the project. A pipeline contains a sequence of tasks which are all related to the specific milestone of the pipeline. You can also use a pipeline task as a standalone project if there is no need for multiple pipelines.
- **Task**: A task is a single unit of work. That should be executed by a single Agent

For more specific use cases we also defined Tasks of type:
- **Issue**: An issue resolving task
- **Subtask**: If we need to break down a task further down. Not in use yet.


# Decomposer: Break down a task into subtasks

The initial building block for task chain is the decomposer. It takes a task and breaks it down into subtasks. This is done by sequence and sometime by an iteration of chains (see [Langchain Chains](https://python.langchain.com/en/latest/modules/chains/getting_started.html)). After decomposing we want to have a tree structure of task objects.

You can choose between a Pipeline Decomposer or Project Decomposer based on your use case. We will use the Simple Pipeline Decomposer in this example to break down a task into a sequence of subtasks.

## Example: Pipeline Decomposer
1. Setup a Task Storage
2. Setup Decomposer
3. Define a objective


In [1]:
from task_chain.storage.context_store import TaskContextStore
from task_chain.decompose.pipeline import SimplePipelineDecomposer

task_storage = TaskContextStore()
decomposer = SimplePipelineDecomposer(task_storage)

objective = "Create a report about the demographics of the population of India"

decomposer.create_tasks(objective, verbose=True)



> Entering new LLMChain chain...
Prompt after formatting:

Breakdown the following task into a sequence of subtasks. Define the input and output keys of each task as lowercase strings.
ROOT TASK: 
Create a report about the demographics of the population of India


Return your respond as string in the following format without any introducing or describing text:

Name of Root Task:
    - Name of Subtask 1
    - Name of Subtask 2
    ...
    - Name of Subtask n
    


Additional instructions:
Create a flat list as shown above do not break subtasks down further.
Try to break down into as few subtasks as possible.
Begin!


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Based on the following sequence of subtasks, define the input and output keys of each task as lowercase strings with underscores and
add a description for each subtask.
TASK LIST: 
root task:
    - gather demographic data of India population
    - analyze demographic data to identify trends a

(Task(id='8e77168e8948459e9bd8bb667c1ae524', name='Create a report about the demographics of the population of India', description='Generate a report summarizing the demographic data collected and the analysis performed on that data.', status=<TaskStatus.OPEN: 'open'>, type=<TaskType.PIPELINE: 'pipeline'>, summary='root task:\n    - gather demographic data of India population\n    - analyze demographic data to identify trends and patterns\n    - create a report summarizing the demographic information and analysis\n\ninput keys:\n- demographic data\n- analysis of demographic data\n\noutput keys:\n- demographic report', details=None, results=None, relations={}, inputs=['demographic data', 'analysis of demographic data'], outputs=['demographic report']),
 [Task(id='2633c23b7e0448d39eea9c70f09afec5', name='gather demographic data of India population', description='Collect data on the population of India, including age, gender, ethnicity, location, and other relevant factors.', status=<Task

We can now view the tree structure of the tasks in the storage

In [2]:
print(task_storage.repr_tree())

┐
└──Create a report about the demographics of the population of India
   ├──gather demographic data of India population
   ├──analyze demographic data to identify trends and patterns
   └──create a report summarizing the demographic information and analysis


As the tree shows the pipeline decomposer creates a root task with a sequence of subtasks. We can access the tasks by their id. Let's first get the root id.

In [3]:
print(task_storage.root_id)

8e77168e8948459e9bd8bb667c1ae524


Now we can access the root task by calling the get_task() method of the storage.

In [4]:
task = task_storage.get_task(task_storage.root_id)
print(task.colored_card_str())

 --------------------------------------------------------------------
| TASK TYPE:    PIPELINE                            STATUS: OPEN     |
|                                                                    |
| ID:           8e77168e8948459e9bd8bb667c1ae524                     |
|                                                                    |
| NAME:         Create a report about the demographics of the pop-   |
|               ulation of India                                     |
| DESCRIPTION:  Generate a report summarizing the demographic dat-   |
|               a collected and the analysis performed on that da-   |
|               ta.                                                  |
| INPUTS:       demographic data, analysis of demographic data       |
| OUTPUTS:      demographic report                                   |
 --------------------------------------------------------------------


Now lets view the subtasks of the root task

In [6]:
subtasks = task_storage.get_children(task_storage.root_id)
for subtask in subtasks:
    print(subtask.colored_card_str())

 --------------------------------------------------------------------
| TASK TYPE:    TASK                                STATUS: OPEN     |
|                                                                    |
| ID:           2633c23b7e0448d39eea9c70f09afec5                     |
|                                                                    |
| NAME:         gather demographic data of India population          |
| DESCRIPTION:  Collect data on the population of India, includin-   |
|               g age, gender, ethnicity, location, and other rel-   |
|               evant factors.                                       |
| INPUTS:                                                            |
| OUTPUTS:      demographic data                                     |
 --------------------------------------------------------------------
 --------------------------------------------------------------------
| TASK TYPE:    TASK                                STATUS: OPEN     |
|        

# Decompose and assign Agents
The main purpose of task decomposition is to break down a complex objective into simple units of work that each can be executed by a single agent. Therefore, we can add a list of available agents to the decomposer. The decomposer will then assign the subtasks to the agents.
The schema for this is similar to the concept of providing tools to langchain agents. Simply by defining a name and a description for each agent.

For the later loading of agents we will have to define register agents to the agent registry. But for now we can simply use a list of dictionaries to define the agents.


In [2]:
agents = [
    {
        "name": "Research_Agent",
        "description": "An agent useful for performing data gathering on the internet"
    },
    {
        "name": "Data_Analysis_Agent",
        "description": "An agent useful for performing data analysis"
    },
    {
        "name": "Data_Visualization_Agent",
        "description": "An agent useful for performing data visualization"
    },
    {
        "name": "Report_Agent",
        "description": "An agent useful for creating reports"
    },
    {
        "name": "Fake_Agent",
        "description": "An agent useful for nothing"
    }
]

## Adding Agents to Decomposer
Now, let's rerun the decomposition while adding the agents to the decomposer.

In [3]:
from task_chain.storage.context_store import TaskContextStore
from task_chain.decompose.pipeline import SimplePipelineDecomposer

task_storage = TaskContextStore()
decomposer = SimplePipelineDecomposer(task_storage, agents=agents)

objective = "Create a report about the demographics of the population of India"

decomposer.create_tasks(objective)

subtasks = task_storage.get_children(task_storage.root_id)
for subtask in subtasks:
    print(subtask.colored_card_str())



> Entering new LLMChain chain...
Prompt after formatting:

Breakdown the following task into a sequence of subtasks. Define the input and output keys of each task as lowercase strings.
ROOT TASK: 
Create a report about the demographics of the population of India


Return your respond as string in the following format without any introducing or describing text:

Name of Root Task:
    - Name of Subtask 1
    - Name of Subtask 2
    ...
    - Name of Subtask n
    


Additional instructions:
Create a flat list as shown above do not break subtasks down further.
Try to break down into as few subtasks as possible.
Begin!


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Based on the following sequence of subtasks, define the input and output keys of each task as lowercase strings with underscores and
add a description for each subtask.
TASK LIST: 
root task:
    - gather demographic data on India's population
    - analyze data to identify key demographic tr

# Persist Tasks
We can persist the tasks in the storage to a file. Because task_chain heavily relies on the task composition, we have all the information and states needed to load or recreate a project, pipeline, etc. within task_storage.

In [4]:
persist_path = "./storage_showcases/demographics_india.json"
task_storage.persist(persist_path)

Persisted task store to ./storage_showcases/demographics_india.json
